# Load and initialize

settings

In [54]:
# dir="~/Desktop/biology/breast_cancer/data/"
# ubunto:
# old samples
dir1="~/Dropbox/CyTOF_Breast/Kaplan_1st/"
# new samples
dir2 = '~/Dropbox/CyTOF_Breast/Kaplan_2nd/data_afterGating/processed data/'
dir3 = '/home/yishai/Dropbox/CyTOF_Breast/Kaplan_3rd/data/'

# parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir+'/functions/')

dir_data = parent_dir+'/Data/'


# show figures (figures are created and saved to file)
show = True
# create plot visualizations
visualize = False
subsample = False



# plt.savefig("test.svg", format="svg")
saveSVG = False

# plot dir
dir_plots = parent_dir+'/Plots_preproccess/'
settings =        (dir_plots,show,saveSVG)





envirement

In [55]:

import time 
start = time.process_time()
import os

import sys
from IPython.display import Image, display
# from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
# import matplotlib.patches as  mpatches
plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

from scipy import signal, stats
import seaborn as sns

from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

from lmfit import minimize, Parameters
# from umap import UMAP

# import scanpy as sc
import anndata

# import schist as scs

# import networkx as nx

# from castle.common import GraphDAG
# from castle.metrics import MetricsDAG
# from castle.datasets import IIDSimulation, DAG
# from castle.algorithms import PC,Notears,GOLEM,ANMNonlinear,DirectLiNGAM,ICALiNGAM,NotearsLowRank
# import notears.notears as notears

# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon

# import keras
# from keras import layers
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

# from statsmodels.graphics._regressionplots_doc import _plot_influence_doc
# from statsmodels.regression.linear_model import OLS
# from statsmodels.stats.outliers_influence import variance_inflation_factor

from pandas import MultiIndex, Int16Dtype
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split

from plot_functions import *
from usefull_functions import *
from preprocess_functions import *
from functions import *
from impute_functions import *
# from normalization import *


%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

folderExists(dir_plots)
folderExists(dir_data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Feature names (Participating panel)

Received CyTOF data is assumed to be only of intact and alive cells without duplicates 

each sample matrix with high dimensionality feature space (NamesAll);
1. 1st cytof batch without: 'NCad','ECad','panKeratin'
2. 2nd cytof batch without: 'p53', 'ZEB1'
3. 3rd cytof batch without: 'p53', 'ZEB1'

un-needed features (removed from  future processing); 
1. 'Live_Dead' (just noise amplification)
2. 'Ir_DNA2'

In [56]:
features = {}
features['NamesAll'] = [  
            'CD45','H4','H3', 'H3.3','NCad','ECad','panKeratin', 'K5', 'EpCam', 'H3K27me2', 'p53', 'EZH2',  
            'gH2AX','aSMA','H3K36me2','H3K4me1','H3K9me2','H4K16ac',
            'H2Aub', 'Vimentin', 'H3K64ac', 'BMI-1', 'ZEB1',  'H3K27ac', 'H4K20me3', 'ER', 'CD49f', 'CD24',
            'GATA3',  'H3K9ac', 'H3K9me3', 'CD44', 'Ki67', 'K8-18', 'H3S28p', 'Ir_DNA2', 'Live_Dead',
            'H3K36me3','H3K4me3','H3K27me3'
        ]
features['Core'] = ['H4','H3', 'H3.3']
# extra cellular are outside the cell - except them everything is noramalized (in group "ToNorm")
features['ToNorm'] = [    
            'H4','H3', 'H3.3','panKeratin', 'K5', 'H3K27me2', 'p53', 'EZH2', 'gH2AX', 'aSMA', 'H3K36me2', 'H3K4me1', 'H3K9me2',
            'H4K16ac', 'H2Aub', 'Vimentin', 'H3K64ac', 'BMI-1', 'ZEB1', 'H3K27ac', 'H4K20me3',
            'ER', 'GATA3', 'H3K9ac', 'H3K9me3', 'Ki67', 'K8-18', 'H3S28p', 'H3K36me3','H3K4me3','H3K27me3',
        ]
features['CellIden'] = [  
            'CD45', 'K5', 'EpCam', 'aSMA', 'Vimentin', 'ZEB1', 'ER', 'CD49f', 'CD24', 'GATA3', 'CD44', 'K8-18',
            'Ki67','p53', 'NCad','ECad','panKeratin',
        ]
# everything that is not cell identity; 'H4','H3', 'H3.3' are excluded
features['EpiCols'] = [   
            'H3K27me2', 'H3K36me2', 'H3K4me1', 'H3K9me2', 'H4K16ac', 'H2Aub', 'H3K64ac',  
            'H3K27ac', 'H4K20me3', 'BMI-1','EZH2','gH2AX',
            'H3K9ac', 'H3K9me3', 'H3S28p','H3K36me3', 'H3K4me3','H3K27me3',
        ]
# remove un-needed features
features = removeFeatures(features.copy(),features =['Ir_DNA2', 'Live_Dead'])
# verify: no feature is both in CellIden and EpiCols (CellIden +EpiCols = NamesAll) 
if test_fetures(features): print (f'features are correct!') 
else:    print ('error')     

features are correct!


Load data from csv, and store in dictionary 

In [57]:
# create dictionarys for data and used feature in each sample:
k={}; names = {}
names['figures'] = [1,1,2,2,2,3,3,3,3]
names['norm_type'] = [1,1,2,2,2,2,2,2,2]
names['all'] = features


# 1st cytof batch
# features diffrence of 1st cytof batch - copy into names dict
f = removeFeatures(features.copy(),features =['NCad','ECad','panKeratin'])
names['1'],names['2']  = f,f
# copy csv data into data dict according to used features dict (names)
k['1'] = pd.read_csv(dir1+"BCK-01_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")[names['1']['NamesAll']]
k['2'] = pd.read_csv(dir1+"BCK-02_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")[names['2']['NamesAll']]

# 2nd cytof batch
# features diffrence of 2nd cytof batch - copy into names dict
f = removeFeatures(features.copy(),features =['p53','ZEB1'])
names['3'],names['4'],names['5']  = f,f,f
k['3'] = pd.read_csv(dir2+"export_BCK03_noaf_23Nov2022_01_0_final_cells - processed.csv")[names['3']['NamesAll']]
k['4'] = pd.read_csv(dir2+"export_BCK04_noaf_23Nov2022_02_0_final_cells - processed.csv")[names['4']['NamesAll']]
k['5'] = pd.read_csv(dir2+"export_BCK05_noaf_23Nov2022_01_0_final_cells - processed.csv")[names['5']['NamesAll']]
# later analysis k6 with duplicates
# K6=pd.read_csv(dir2+"export_BCK05_noaf_23Nov2022_01_0_final_cells.csv")

# 3d cytof batch ADDED H3K4me33
# features diffrence of 3d cytof batch - copy into names dict
f = removeFeatures(features.copy(),features =['p53','ZEB1'])
names['4.1'],names['7'],names['8'],names['11']  = f,f,f,f
k['4.1'] = pd.read_csv(dir3+"export_BCK04_noaf_16Mar2023_03_0_FinalCells.csv")[names['4.1']['NamesAll']]
k['7'] = pd.read_csv(dir3+"export_BCK07_noaf_16Mar2023_03_0_FinalCells.csv")[names['7']['NamesAll']]
k['8'] = pd.read_csv(dir3+"export_BCK08_noaf_16Mar2023_02_0_FinalCells.csv")[names['8']['NamesAll']]
k['11'] = pd.read_csv(dir3+"export_BCK11_noaf_16Mar2023_01_0_FinalCells.csv")[names['11']['NamesAll']]

# gating and transformation



gating - fix fluidigm gating result; In-gate and outlier removal (Samples with low number of events)

Samples with low number of events are discarded (outliers) - remove outlier 99.99% from all

In gate to achieve good normalization - gate on H3.3/H4 was too low


In [58]:

GateColumns=['H3.3','H4']#,'H3']#,'H3']
for [i, K] , j in zip(k.items(), names['norm_type']):
    initialSize = len(K)
    if j==1:
      K = Gate(K,'k' + i,['H3.3','H4'])
    if j==2:
      K = Gate(K,'k' + i,['H3','H3.3','H4'])
    finalSize = len(K)
    print (f'k{i}: gated with method {j}, initial size: {initialSize}, after gating and outliers: {finalSize} ({np.round(finalSize/initialSize*100,2)}%)')
    k[i] = K.reset_index()
    

# k_gate = k

k1: gated with method 1, initial size: 6195, after gating and outliers: 4811 (77.66%)
k2: gated with method 1, initial size: 299471, after gating and outliers: 181275 (60.53%)
k3: gated with method 2, initial size: 3968, after gating and outliers: 815 (20.54%)
k4: gated with method 2, initial size: 401141, after gating and outliers: 374890 (93.46%)
k5: gated with method 2, initial size: 79376, after gating and outliers: 68765 (86.63%)
k4.1: gated with method 2, initial size: 111614, after gating and outliers: 110463 (98.97%)
k7: gated with method 2, initial size: 70379, after gating and outliers: 66741 (94.83%)
k8: gated with method 2, initial size: 130074, after gating and outliers: 124725 (95.89%)
k11: gated with method 2, initial size: 41305, after gating and outliers: 38864 (94.09%)


impute

impute EpiCols with the right dist

In [59]:

# n = names['all']['EpiCols'].copy()
# remove cropped distrunutions
# for f in ['H3K4me1','H3K4me3','H3K27me2','H3K36me2','H3S28p']:
#     n.remove(f)
# zeros = getZerosMat(k,index=['1','2','3','4','5'],columns = n)

zeros = getZerosMat(k,columns = names['all']['NamesAll'].copy(),percent = 0)
zeros.to_csv(settings[0]+'raw_zeros.csv')
# print(zeros)
# # m=
# # 

In [60]:
# def plot_hist(k,NamesAll,figures,settings,func = sns.kdeplot ,title = '',Figname = '' ):
    
#     arr = np.linspace(0, 1, len (k.keys()))    
#     colors = cm.rainbow(arr)
#     for M in NamesAll: 
#         fig, ax = plt.subplots(1,2,figsize=(10,4))
        
#         for [i, K],color,fig_num in zip(k.items(),colors,figures):
#             fig_num -= 1
#             ax[fig_num].set_ylim(0,5000)
#             ax[fig_num].set_xlim(0,500)

#             # sns.kdeplot(K[M],c=c,label='Tumor ' + i)
#             try: #if K doesnt contain the feature pass..
#               q=func(K[M],color=color,label='Tumor ' + i,ax = ax[fig_num])
#               # sns.kdeplot(K2[M],c='g',label='Tumor 2')
#               ax[fig_num].title.set_text(title)
#               ax[fig_num].legend()
              
              
#             except:
#               pass
        
#         figname = Figname + M
        
#         dir,show,saveSVG = settings
#         plt.savefig(dir+figname+'.png', format="png", bbox_inches="tight", pad_inches=0.2)
#         if saveSVG:
#             plt.savefig(dir+figname+'.svg', format="svg", bbox_inches="tight", pad_inches=0.2)
#         if show:
#             plt.show()
#         else:
#             plt.close()


In [61]:
# i='1'
# n=zeros.T[zeros.T['1'].notnull()].index.tolist()

# nonzeros = MimalzerosOverlap(k[i].copy(),names[i]['Core']+names[i]['CellIden'],n)
# features = nonzeros.columns
# K = k[i].copy()
# # features=['p53']
# df = pd.DataFrame(index=['mse','mae','accuracy'],columns = features)

# for feature in features:

#     K,mse, mae, accuracy = xg(nonzeros,K,feature)
#     df.at['mse',feature] = mse
#     df.at['mae',feature] = mae
#     df.at['accuracy',feature] = accuracy



In [62]:
# print(zeros.T[zeros.T['1'].notnull()].index.tolist())
# print(df.T.index)
# print(df.T)

In [63]:
# print(df.T) 
# print(K)

save the raw data

In [64]:
# for i, K in k.items():
#   dict ={}
#   dict['k'] = K
#   dict['names'] = names[i]
#   dict['all'] = names['all']
#   pickle_dump('k_raw'+i+'_dict', dict,dir_data)
#   print('k'+i,'; saved to file')

 arcsinh data transformation
  
  transformation (semi log scaling): scale argument = 5

In [65]:
for i, K in k.items():
    k[i] = arcsinh_transform(K)
    print ('k' + i,'; arcsinh transformed')

k1 ; arcsinh transformed
k2 ; arcsinh transformed
k3 ; arcsinh transformed
k4 ; arcsinh transformed
k5 ; arcsinh transformed
k4.1 ; arcsinh transformed
k7 ; arcsinh transformed
k8 ; arcsinh transformed
k11 ; arcsinh transformed


visualization

Kernel Distribution Estimation Plot (kdeplot) is the probability density function plot;
1. can plot for the univariate (single variable) or multiple variables altogether. 
2. y = probability, x = feature value after arcsinh transformation (can be also negative)
3. arcsinh probability density plot - analysis of herogeneity (division to multiple populations)

results:
1. cd45 - we want only the negative population
2. close populations - epcam, vimentin,cd49f
3. far populations - cd44

In [66]:
if visualize:
#   interst=[  'CD45','CD44', 'EpCam', 'Vimentin', 'CD49f','CD45','H4','H3', 'H3.3','NCad','ECad','panKeratin',]
  interst=names['all']['NamesAll']
  plot_hist(k,interst,names['figures'],settings,
            func = sns.kdeplot,title = 'ArcSinh Unnormalized',Figname = '1_Hist_ArcSinh_Unnormalized_' ) 

# Split CD45 neg 

the only interesting population

In [67]:


KCD45Neg_ind ,min_x, min_y = {},{},{}

for [i, K] , xMin in zip(k.items(),[1.87,2.172,0.98,0.06,1.2298199181170908,0.073,0.12,0.11,0.15]):
    if xMin == None: #there  are 2 defined dist and minima can be found not manually
         min_x[i],min_y[i] = splitInversePDF(K,i,'CD45')
    else:
        min_x[i] = xMin
        if visualize:
            min_x[i],min_y[i] = splitInversePDF(K,i,'CD45',min_x = xMin)
    KCD45Neg_ind[i]= K['CD45']<min_x[i]
    print(f'k{i}; value with highest *inverse* probability:', min_x[i])

if visualize:
    for i, K in k.items():
        plotSplit(K,i,min_x[i],min_y[i],settings,Figname = '_cd45_split',log = False)
    


k1; value with highest *inverse* probability: 1.87
k2; value with highest *inverse* probability: 2.172
k3; value with highest *inverse* probability: 0.98
k4; value with highest *inverse* probability: 0.06
k5; value with highest *inverse* probability: 1.2298199181170908
k4.1; value with highest *inverse* probability: 0.073
k7; value with highest *inverse* probability: 0.12
k8; value with highest *inverse* probability: 0.11
k11; value with highest *inverse* probability: 0.15


In [68]:
# if visualize:
#   colors = cm.rainbow(np.linspace(0, 1, len (k.keys())))
#   fig, ax = plt.subplots(1,2,figsize=(10,3))
#   for i, K in k.items(): 
#     #   
#     j =  names['figures'][int(i)-1]-1 
#     sns.histplot(K.CD45,color=colors[int(i)-1],label='T'+i,stat='density',element='step',fill=False,ax = ax[j])
#     ax[j].legend(loc='upper center',bbox_to_anchor=(1,1))
#     ax[j].set_yscale('log')
#     ax[j].title.set_text('cd45 split histplot')
#   plt.savefig(dir_plots+'cd45_split_histplot.svg', format="svg", bbox_inches="tight", pad_inches=0.2)

 


#   fig, ax = plt.subplots(1,2,figsize=(12,3))
#   for [i, K],[h, KCD45Neg] in zip( k.items(),kCD45Neg.items()):
#       j =  names['figures'][int(i)-1]-1 
#       sns.kdeplot(K.CD45,color=colors[int(i)-1],label='T'+i,ax = ax[j])
#       sns.kdeplot(K.CD45,color=colors[int(i)-1],ls='--',label='T'+i+' CD45-',ax = ax[j])
#       ax[j].legend(loc='upper center',bbox_to_anchor=(1,1))
#       ax[j].set_yscale('log')
#       ax[j].title.set_text('cd45 split kdeplot')
#   plt.savefig(dir_plots+'cd45_split_kdeplot.svg', format="svg", bbox_inches="tight", pad_inches=0.2)

      

Continue only with CD45 neg population

1 - delete cd45+ population and continue with k  = kCD45Neg

2 - delete cd45 feature from data 

In [69]:

dict ={}
for i, K in k.items():
    if i != '3':
        newK = K[KCD45Neg_ind[i]].copy().reset_index()
        print(f'k{i}: CD45+ removal - initial: {len(K)}, final:{len(newK)} ({np.round(len(newK)/len(K)*100,2)}%)')
        dict[i ] = newK
k =None;newK=None; k=dict 

dict ={}
for i, K in k.items():
  

  # remove from features lists(names)
  N = names[i]
  for j, sublist in N.items():
    try:
      sublist.remove('CD45')
    except:
      pass
    N[j] = sublist
  dict[i] = N
#   remove index 3 and copy to new dict
names['figures'].pop(3-1)
dict['figures'] = names['figures']
names['norm_type'].pop(3-1)
dict['norm_type'] = names['norm_type']
dict['all'] = names['all']
del names; names=dict 

k1: CD45+ removal - initial: 4811, final:1772 (36.83%)
k2: CD45+ removal - initial: 181275, final:39276 (21.67%)
k4: CD45+ removal - initial: 374890, final:313781 (83.7%)
k5: CD45+ removal - initial: 68765, final:63390 (92.18%)
k4.1: CD45+ removal - initial: 110463, final:99457 (90.04%)
k7: CD45+ removal - initial: 66741, final:57348 (85.93%)
k8: CD45+ removal - initial: 124725, final:108977 (87.37%)
k11: CD45+ removal - initial: 38864, final:33196 (85.42%)


# Normalize using new method on all intercellular markers

save pre normalized data

In [70]:
for i, K in k.items():
    try:
      del dict;dict ={}
    except:
        dict ={}
    dict['k'] = K
    pickle_dump('k'+i+'_prenorm_dict', dict,dir_data)

In [71]:
# uncommonFeatures = ['NCad','ECad','panKeratin',  'p53', 'ZEB1' ]


# # remove from the  mutual list uncommonFeatures
# N = names['1'].copy()

# for j, sublist in N.items():
#     for f in uncommonFeatures:
#         try:
#             sublist.remove(f)
#         except:
#             pass
#     N[j] = sublist
# names['1245'] = N

# names['figures'].append(2)


#  # remove from data
# appendDict ={}
# for i, KK in k.items():
#     K=KK.copy()
#     for f in uncommonFeatures:
#         try:
#             K = K.drop(columns=[f])
#         except:
#             pass
#     # print (K.columns)
#     appendDict[i ] = K
    
# # del k; k=dict 

# # append data
# NamesAll = names['1245']['NamesAll']
# k_append= pd.DataFrame(columns =NamesAll)
# for i, K in appendDict.items():

#     K= subsample_k(K[NamesAll].copy(),n=1771)
#     K['by_sample'] = int(i)
#     k_append = k_append.append(K, ignore_index=True)
# by_sampleInd = k_append['by_sample'].copy()
# k['1245'] = k_append

# k['1245']['by_sample'] = by_sampleInd
# # k_append['by_sample'] = by_sampleInd
# # print(len(by_sampleInd))

2 step normalization:

1- normalize gardient (TBD)

1- normalize using core

insight: since its the minima multiple rus with same effect as one run

note: features to norm (i.e names[i]['ToNorm']) are different on each sample

In [72]:
# if visualize:
#   for i, K in k.items():
#     print ('k' + i,', initial std:')
#     print(K.std().to_frame().T)




def normalize_data(k,sample,names):
    for [i, K] , j in zip(k.items(), names['norm_type']):
        
        
        # FIRST STEP - normalize_gardient by core 
        # SECOND STEP - normalize ToNorm fetures by the core features ['H3.3','H4']
        
        if j==1:
          K=NormalizeNew2(K,names[i]['ToNorm'])
          K = Mean_Core_normalization(K, names[i]['ToNorm'],coreFetures=['H3.3','H4'])
          
        elif j==2:
          K=NormalizeNew(K,names[i]['ToNorm'])
          K = Mean_Core_normalization(K, names[i]['ToNorm'],coreFetures=['H4','H3', 'H3.3'])
        else: 
          print ('error')
          break
        
        k[i] = K 
        print (sample + i,f'; normalized with method {j}')
    return k

k = normalize_data(k,'k',names)





k1 ; normalized with method 1
k2 ; normalized with method 1
k4 ; normalized with method 2
k5 ; normalized with method 2
k4.1 ; normalized with method 2
k7 ; normalized with method 2
k8 ; normalized with method 2
k11 ; normalized with method 2


# merged datasets and subsampling
create a merged to single dataset
1. samples 1245
2. samples 12456789

In [73]:
for i, K in k.items():
    k[i]['by_sample'] = float(i)
    names[i]['ind'] = k[i]['by_sample']

# merged dataset is normalize with H3 and plot by its own fig
names['norm_type'].append(2)
names['figures'].append(4)


uncommonFeatures1245 = ['NCad','ECad','panKeratin',  'p53', 'ZEB1' ]
uncommonFeatures456789 = ['panKeratin',  'p53', 'ZEB1' ]
uncommonFeatures12456789 = ['NCad','ECad','panKeratin',  'p53', 'ZEB1']

# remove the uncommonFeatures from the  mutual list, create data-dict without uncommonFeatures and append into single dataset
names['1245']  = removeFeatures(names['all'].copy(),uncommonFeatures1245)
k['1245'],names['1245']['ind'] = createAppendDataset(names['1245'],getAppendDict(k,['1','2','4','5'],uncommonFeatures1245 ),n=1771)

names['456789']  = removeFeatures(names['all'].copy(),uncommonFeatures456789)
k['456789'],names['456789']['ind']= createAppendDataset(names['456789'],getAppendDict(k,['4','5','4.1','7','8','11'],uncommonFeatures456789 ),n=5000)

names['12456789']  = removeFeatures(names['all'].copy(),uncommonFeatures12456789)
k['12456789'],names['12456789']['ind']= createAppendDataset(names['12456789'],getAppendDict(k,['1','2','4','5','4.1','7','8','11'],uncommonFeatures12456789 ),n=1771)



original size: 1772, new size: 1771
original size: 39276, new size: 1771
original size: 313781, new size: 1771
original size: 63390, new size: 1771


/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)


original size: 313781, new size: 5000
original size: 63390, new size: 5000
original size: 99457, new size: 5000
original size: 57348, new size: 5000
original size: 108977, new size: 5000
original size: 33196, new size: 5000


/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai

original size: 1772, new size: 1771
original size: 39276, new size: 1771
original size: 313781, new size: 1771
original size: 63390, new size: 1771
original size: 99457, new size: 1771
original size: 57348, new size: 1771
original size: 108977, new size: 1771
original size: 33196, new size: 1771


/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai/breast_cancer/functions/usefull_functions.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k_append = k_append.append(K, ignore_index=True)
/home/yishai

save pre-scaled data to file

In [74]:
# for i, K in k.items():
#     try:
#       del dict;dict ={}
#     except:
#         dict ={}
#     dict['k'] = K
#     pickle_dump('k'+i+'_prescaled_dict', dict,dir_data)

# Scale data 

using mean, std (whiten data) - achieve std close to 1 mean close to 0

In [75]:

for i, K in k.items():

    k[i] = scale_data(K)
    print (f'k{i} scaled')
    # print(K.std().to_frame().T)
# the index numbers are unfortunately passes into scaling and norm
# after scaling rturn the unscled index numbers
for i, K in k.items():
    k[i]['by_sample'] = names[i]['ind']
    names[i]['sampleInd'] = None
    


k1 scaled
k2 scaled
k4 scaled
k5 scaled
k4.1 scaled
k7 scaled
k8 scaled
k11 scaled
k1245 scaled
k456789 scaled
k12456789 scaled


In [76]:


if visualize:
    plot_hist(k,names['all']['NamesAll'],names['figures'],settings,
        func = sns.kdeplot,title = 'Normalized + Scaled',Figname = '2_Hist_normalized_scaled_' )     

subsample original data; verify n<=5000 

In [77]:
if subsample:
    for i in list(k.keys())[:8]:
        k[i] = subsample_k(k[i],n=5000)

list(k.keys())[:8]

['1', '2', '4', '5', '4.1', '7', '8', '11']

## other
Save data to file (pickle)

convert images to pdf


In [78]:
for i, K in k.items():
  print ('k'+i+ '; samples = ', len(K), ', fetures = ', len(K.columns))

for i, K in k.items():
    try:
      del dict;dict ={}
    except:
        dict ={}
    dict['k'] = K
    pickle_dump('k'+i+'_dict', dict,dir_data)
    
    del dict;dict ={}
    dict = names[i]
    # dict['all'] = names['all']
    pickle_dump('k'+i+'_names_dict', dict,dir_data)
    print('k'+i,'; saved to file')



end = time.process_time()
print ('total run time =', end-start )

k1; samples =  1772 , fetures =  38
k2; samples =  39276 , fetures =  38
k4; samples =  313781 , fetures =  39
k5; samples =  63390 , fetures =  39
k4.1; samples =  99457 , fetures =  39
k7; samples =  57348 , fetures =  39
k8; samples =  108977 , fetures =  39
k11; samples =  33196 , fetures =  39
k1245; samples =  7084 , fetures =  34
k456789; samples =  30000 , fetures =  36
k12456789; samples =  14168 , fetures =  34
k1 ; saved to file
k2 ; saved to file
k4 ; saved to file
k5 ; saved to file
k4.1 ; saved to file
k7 ; saved to file
k8 ; saved to file
k11 ; saved to file
k1245 ; saved to file
k456789 ; saved to file
k12456789 ; saved to file
total run time = 29.793874500000015
